# Знакомство с word2vec


## Загрузка модели
Скачаем модель <code>google-news-vectors</code>. Откроем ее с помощью библиотеки <code>gensim</code>.

In [1]:
! pip install -q -U gensim
! pip install -q opendatasets
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300")

# Для быстрой загрузки данных следует ввест логин и токен с Kaggle
# либо убрать последнюю строку загружать файл локально в колаб (долго)



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: maxjul
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300


100%|██████████| 3.17G/3.17G [00:35<00:00, 96.6MB/s]


In [3]:
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim.models import KeyedVectors

w = KeyedVectors.load_word2vec_format("googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin",
                                      binary=True)

Структура называется <code>KeyedVectors</code> и по сути представляет собой отображение между ключами и векторами. Каждый вектор идентифицируется своим ключом поиска, чаще всего коротким строковым токеном, поэтому обычно это соответствие между

<center><code>{str => 1D numpy array}</code></center><br/>



Например, выведем первые 10 координат вектора, соответствующего слову <code>sunrise</code>

In [4]:
print("Размерность вектора: ", w["sunrise"].shape)
print("Первые 10 координат вектора: \n", w["sunrise"][:10])

Размерность вектора:  (300,)
Первые 10 координат вектора: 
 [-0.22558594 -0.03540039 -0.21679688  0.03613281 -0.2265625  -0.09814453
  0.109375   -0.34570312  0.18652344  0.01806641]


## Задание 1. Сходство.

Извлеките векторы слов <code>London</code>, <code>England</code>, <code>Moscow</code>. Посчитайте косинусное расстояние между словами <code>London</code> и <code>England</code> и между словами <code>Moscow</code> и <code>England</code>. Какая пара слов ближе? Подсказка: для вычисления косинусного расстояния использвется метод <code>distance()</code>. Правильный ответ представлен в блоке вывода.

In [6]:
help(w.distance)

Help on method distance in module gensim.models.keyedvectors:

distance(w1, w2) method of gensim.models.keyedvectors.KeyedVectors instance
    Compute cosine distance between two keys.
    Calculate 1 - :meth:`~gensim.models.keyedvectors.KeyedVectors.similarity`.
    
    Parameters
    ----------
    w1 : str
        Input key.
    w2 : str
        Input key.
    
    Returns
    -------
    float
        Distance between `w1` and `w2`.



In [11]:
print(f"London = > England:  {w.distance('London', 'England')}")
print(f"Moscow = > England:  {w.distance('Moscow', 'England')}")

London = > England:  0.5600714385509491
Moscow = > England:  0.8476868271827698


## Задание 2. Аналогии.
С помощью метода most_similar решите аналогию
```London : England = Moscow : X```

Правильный ответ представлен в блоке вывода.

(Подсказка: нужно использовать аргументы positive и negative)

In [12]:
help(w.most_similar)

Help on method most_similar in module gensim.models.keyedvectors:

most_similar(positive=None, negative=None, topn=10, clip_start=0, clip_end=None, restrict_vocab=None, indexer=None) method of gensim.models.keyedvectors.KeyedVectors instance
    Find the top-N most similar keys.
    Positive keys contribute positively towards the similarity, negative keys negatively.
    
    This method computes cosine similarity between a simple mean of the projection
    weight vectors of the given keys and the vectors for each key in the model.
    The method corresponds to the `word-analogy` and `distance` scripts in the original
    word2vec implementation.
    
    Parameters
    ----------
    positive : list of (str or int or ndarray) or list of ((str,float) or (int,float) or (ndarray,float)), optional
        List of keys that contribute positively. If tuple, second element specifies the weight (default `1.0`)
    negative : list of (str or int or ndarray) or list of ((str,float) or (int,floa

In [18]:
w.most_similar(positive=['England', 'Moscow'], negative=['London'], topn=10)

[('Russia', 0.6502718329429626),
 ('Ukraine', 0.5879061818122864),
 ('Belarus', 0.5666376352310181),
 ('Azerbaijan', 0.5418694615364075),
 ('Armenia', 0.5300518870353699),
 ('Poland', 0.5253247618675232),
 ('coach_Georgy_Yartsev', 0.5220180749893188),
 ('Russian', 0.5214669108390808),
 ('Croatia', 0.5166040658950806),
 ('Moldova', 0.5125792026519775)]

## Задание 3. Сходство: найти лишнее.
С помощью метода <code>doesnt_match</code> найдите лишнее слово в ряду <code>breakfast cereal dinner lunch</code>.

Правильный ответ представлен в блоке вывода.

In [19]:
txt_lst = "breakfast cereal dinner lunch".split(" ")
w.doesnt_match(txt_lst)

'cereal'

## Задание 4. Представление предложений в виде векторов


Дано предложение: <code>the quick brown fox jumps over the lazy dog</code>. Вам нужно представить это предложение в виде вектора. Для этого найдите вектор каждого слова в модели, а затем усредните векторы покомпонентно.


In [34]:
import numpy as np

txt = "the quick brown fox jumps over the lazy dog"
w_lst = txt.split(" ")
w_vectors = [w[word] for word in w_lst]

ans = sum(w_vectors)/len(w_vectors)
ans[:5]

array([ 0.09055582,  0.05434163, -0.06713867,  0.10968696, -0.01060655],
      dtype=float32)

# Сравнение двух моделей

## Загрузка ещё одной модели


Откроем модель google-news-vectors и модель, обученную на британском национальном корпусе http://vectors.nlpl.eu/repository/20/0.zip, с помощью gensim.


In [35]:
! wget -c http://vectors.nlpl.eu/repository/20/0.zip
! unzip 0.zip
! head -3 model.txt

--2024-05-10 13:10:16--  http://vectors.nlpl.eu/repository/20/0.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344050746 (328M) [application/zip]
Saving to: ‘0.zip’

0.zip               100%[===================>] 328.11M  24.6MB/s    in 14s     

2024-05-10 13:10:31 (23.1 MB/s) - ‘0.zip’ saved [344050746/344050746]

Archive:  0.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  
163473 300
say_VERB -0.008861 0.097097 0.100236 0.070044 -0.079279 0.000923 -0.012829 0.064301 -0.029405 -0.009858 -0.017753 0.063115 0.033623 0.019805 0.052704 -0.100458 0.089387 -0.040792 -0.088936 0.110212 -0.044749 0.077675 -0.017062 -0.063745 -0.009502 -0.079371 0.066952 -0.070209 0.063761 -0.038194 -0.046252 0.049983 -0.094985 -0.086341 0.024665 -0.

Загрузим модель, обученную на британском национальном корпусе

In [36]:
w_british = KeyedVectors.load_word2vec_format("model.bin", binary=True)

Заметим, что размерность векторов в этом случае также равна 300. При этом через нижнее подчеркивание нужно указывать часть речи используемого слова. Слова следует приводить к нижнему регистру.

In [37]:
try:
    print(w_british["London_NOUN"].shape)
    print('upper is ok')
except:
    print(w_british["london_NOUN"].shape)
    print('lower is ok')

(300,)
lower is ok


## Набор данных для оценки качества
Скачаем датасет wordsim353.



In [38]:
! wget -c http://alfonseca.org/pubs/ws353simrel.tar.gz
! tar -xvf ws353simrel.tar.gz
! head -5 wordsim353_sim_rel/wordsim_similarity_goldstandard.txt

--2024-05-10 13:11:22--  http://alfonseca.org/pubs/ws353simrel.tar.gz
Resolving alfonseca.org (alfonseca.org)... 162.215.249.67
Connecting to alfonseca.org (alfonseca.org)|162.215.249.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5460 (5.3K) [application/x-gzip]
Saving to: ‘ws353simrel.tar.gz’

ws353simrel.tar.gz  100%[===================>]   5.33K  --.-KB/s    in 0s      

2024-05-10 13:11:22 (352 MB/s) - ‘ws353simrel.tar.gz’ saved [5460/5460]

wordsim353_sim_rel/wordsim353_agreed.txt
wordsim353_sim_rel/wordsim353_annotator1.txt
wordsim353_sim_rel/wordsim353_annotator2.txt
wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt
wordsim353_sim_rel/wordsim_similarity_goldstandard.txt
tiger	cat	7.35
tiger	tiger	10.00
plane	car	5.77
train	car	6.31
television	radio	6.77


## Подготовка эталонной выборки


Из файла `wordsim_similarity_goldstandard.txt` извлечем пары слов и посчитаем косинусное сходство их векторов в обеих моделях. Посчитаем корреляцию оценок сходства в модели google-news-vectors с оценками аннотаторов в датасете, а затем - корреляцию сходства в модели на основе британского национального корпуса с оценками аннотаторов в датасете. Какая модель ближе к суждениям экспертов-разметчиков?

(используем только те слова из wordsim, для которых находятся векторы на британском корпусе, помеченные как существительные!)

In [39]:
import pandas as pd

df = pd.read_csv("wordsim353_sim_rel/wordsim_similarity_goldstandard.txt",
                 sep="\t", header=None)
df.columns = ["first", "second", "score"]
df.head(3)

,first,second,score
0,tiger,cat,7.35
1,tiger,tiger,10.00
2,plane,car,5.77


## Вычисление оценок similarity моделей
Используем только те слова из wordsim, для которых находятся векторы на британском корпусе, помеченные как существительные, сформируйте 3 массива с оценкам схожести:

1. Оценки (косинус между векторами), полученные в результате модели google-news-vectors

2. Оценки (косинус между векторами) полученные в результате модели на основе британского национального корпуса

3. Эталонные оценки из word_sim, для слов из которых находятся векторы на британском корпусе

Пропущенные слова из word_sim представлены в блоке вывода.

In [40]:
from scipy.spatial import distance

gn_dist, br_dist, scores = [], [], []

for row in df.iterrows():

  w1, w2 = row[1]["first"], row[1]["second"]

  try:
    w1_br, w2_br = w_british[w1.lower()+"_NOUN"], w_british[w2.lower()+"_NOUN"]
    w1_gn, w2_gn = w[w1], w[w2]

    br_dist.append(distance.cosine(w1_br, w2_br))
    gn_dist.append(distance.cosine(w1_gn, w2_gn))
    scores.append(row[1]["score"])

  except KeyError as e:
    print(e, "Skipping this word.")

"Key 'stupid_NOUN' not present" Skipping this word.
"Key 'arafat_NOUN' not present" Skipping this word.
"Key 'harvard_NOUN' not present" Skipping this word.
"Key 'mexico_NOUN' not present" Skipping this word.
"Key 'live_NOUN' not present" Skipping this word.
"Key 'seven_NOUN' not present" Skipping this word.
"Key 'five_NOUN' not present" Skipping this word.
"Key 'mars_NOUN' not present" Skipping this word.


## Выбор модели: корреляция с экспертами

Вычислите корреляцию Спирмена для каждой модели по сравнению с эталонными оценками из word_sim.

Результаты представлены в блоке вывода.

In [44]:
from scipy.stats import spearmanr

#enter your code here
print(f"GN spearmanr corr: {-spearmanr(scores, gn_dist).statistic}")
print(f"British spearmanr corr: {-spearmanr(br_dist, scores).statistic}")

GN spearmanr corr: 0.7817164245392593
British spearmanr corr: 0.762755193448961


Можно заметить, что модель google-news-vectors несколько выигрывает в данном случае.

# Индивидуальное задание

Определите косинусное расстояние между векторами слов: vodka и drink

In [45]:
w1, w2 = "vodka", "drink"
w1_br, w2_br = w_british[w1.lower()+"_NOUN"], w_british[w2.lower()+"_NOUN"]
w1_gn, w2_gn = w[w1], w[w2]

print("br: ", distance.cosine(w1_br, w2_br))
print("gn: ", distance.cosine(w1_gn, w2_gn))

br:  0.41066306829452515
gn:  0.428485631942749


2. Дан набор слов: vodka drink car gem, определите лишнее слово.

In [48]:
txt_lst = "vodka drink car gem".split(" ")
print("gn: ", w.doesnt_match(txt_lst))
print("br: ", w_british.doesnt_match([word.lower()+"_NOUN" for word in txt_lst]))

gn:  gem
br:  gem_NOUN


 Определите косинусное расстояние между векторами предложений:

Дисклеймер: из оригинальных пословиц удлаены слова, отсутствующие в модели GN.

empty vessel makes much noise

и

idle brain is the devil workshop

In [51]:
import numpy as np

txt1, txt2 = "empty vessel makes much noise", "idle brain is the devil workshop"
w1_lst, w2_lst = txt1.split(" "), txt2.split(" ")
w1_vectors, w2_vectors = [w[word] for word in w1_lst], [w[word] for word in w2_lst]

ans1, ans2 = sum(w1_vectors)/len(w1_vectors), sum(w2_vectors)/len(w2_vectors)

print("gn: ", distance.cosine(ans1, ans2))

gn:  0.5722963213920593


Из набора данных word_sim извлеките подвыборку пар слов с индексами 28:128 (нумерация начинается с нуля, правая граница не включается).

Из уже выбранных пар используйте только те, для слов которых находятся векторы на британском корпусе, помеченные как существительные! Иначе удаляйте такую пару из подвыборки.

Вычислите корреляцию Спирмена между оценками схожести выбранных пар слов, полученных в результате работы моделей, и оценками аннотаторов в датасете word_sim.

In [53]:
df[28:128]

,first,second,score
28,journey,voyage,9.29
29,boy,lad,8.83
30,coast,shore,9.10
31,asylum,madhouse,8.87
32,magician,wizard,9.02
...,...,...,...
123,attempt,peace,4.25
124,consumer,confidence,4.13
125,start,year,4.06
126,focus,life,4.06


In [54]:
from scipy.spatial import distance

gn_dist, br_dist, scores, deleted = [], [], [], 0

for row in df[28:128].iterrows():

  w1, w2 = row[1]["first"], row[1]["second"]

  try:
    w1_br, w2_br = w_british[w1.lower()+"_NOUN"], w_british[w2.lower()+"_NOUN"]
    w1_gn, w2_gn = w[w1], w[w2]

    br_dist.append(distance.cosine(w1_br, w2_br))
    gn_dist.append(distance.cosine(w1_gn, w2_gn))
    scores.append(row[1]["score"])

  except KeyError as e:
    print(e, "Skipping this word.")
    deleted += 1

"Key 'harvard_NOUN' not present" Skipping this word.
"Key 'mexico_NOUN' not present" Skipping this word.


In [55]:
from scipy.stats import spearmanr

#enter your code here
print(f"GN spearmanr corr: {-spearmanr(scores, gn_dist).statistic}")
print(f"British spearmanr corr: {-spearmanr(br_dist, scores).statistic}")
print("deleted:", deleted)

GN spearmanr corr: 0.7067540249210923
British spearmanr corr: 0.6727177121061954
deleted: 2
